In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/plenoi/CMU_DataScience/master/lung.csv',header = None )

df

In [ ]:
df.shape

In [ ]:
df.isnull().sum(axis=0)

In [ ]:
pel_class = np.unique(df[0])
pel_class

In [ ]:
y = df[0].values
X = df.drop([0], axis = 1).values

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=1)
X,y = sm.fit_resample(X,y)

In [ ]:
np.shape(y)

In [ ]:
df.shape

In [ ]:
np.shape(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
fs = SelectKBest(chi2, k=3)
fs.fit(X_train_norm,y_train)
X_train = fs.transform(X_train_norm)
X_test = fs.transform(X_test_norm)


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
params = {
    'C' : [2,4,6,8,10,12], 
    'gamma' : [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32] 
    
}
clf = GridSearchCV(SVC(),params, cv=10)
clf.fit(X_train_norm, y_train)
print("Best params : " + str(clf.best_params_))
print("10CV accuracy : "+str(clf.best_score_*100))

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
y_predict = clf.predict(X_test_norm)
print("Test accuracy : "+str(sum(y_test == y_predict)/len(y_test)*100))

In [ ]:
X_test_norm.shape

In [ ]:
clf

In [ ]:
df_test = pd.read_csv('https://raw.githubusercontent.com/plenoi/CMU_DataScience/master/lung_test.csv?fbclid=IwAR1YAR9tMp-8Su5rr1_jgb_fETkatEQUDDLI9aDjUANLoo3QAmnm8ZMFx_A',header = None )



In [ ]:
yt = df_test[0].values
Xt = df_test.drop([0],axis = 1).values


In [ ]:
X_test = scaler.transform(Xt)

In [ ]:
y_predict = clf.predict(X_test)
print("Test accuracy : "+str(sum(yt== y_predict)/len(yt)*100))

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
eclf = VotingClassifier(
estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],
voting='soft')
params = {'lr__C': [1.0, 100.0], 'rf__n_estimators': [20, 200]}
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
grid = grid.fit(X_train_norm, y_train)

In [ ]:
y_predict = grid.predict(X_test_norm)
print("Test accuracy : "+str(sum(y_test == y_predict)/len(y_test)*100))

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
eclf = StackingClassifier(
estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],
stack_method = 'predict_proba',
final_estimator=LogisticRegression(random_state=1))
params = {'lr__C': [1.0, 100.0], 'rf__n_estimators': [20, 200]}
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
grid = grid.fit(X_train_norm, y_train)

In [ ]:
y_predict = grid.predict(X_test_norm)
print("Test accuracy : "+str(sum(y_test == y_predict)/len(y_test)*100))